In [17]:
import json
import pandas as pd
import numpy as np

In [9]:
# load table & mobie dataset
table = pd.read_csv('final_name_map.csv', sep='\t')  # type: ignore
with open('/home/hellgoth/Documents/work/projects/culture-collections_project/culture-collections/data/new_names/dataset.json', 'r') as f:
    dataset = json.load(f)

In [10]:
table

,Filename,new_filename_iteration4_manualRename,remove,size_c
0,20230616_RCC_6731_pfa_OD_Hoechst-tub-nhs_series00,rcc6731_pfa_dna-tub-nhs_20230616_od_s00,NaN,3
1,20230616_RCC_6731_pfa_OD_Hoechst-tub-nhs_series01,rcc6731_pfa_dna-tub-nhs_20230616_od_s01,NaN,3
2,20230616_RCC_6731_pfa_OD_Hoechst-tub-nhs_series02,rcc6731_pfa_dna-tub-nhs_20230616_od_s02,NaN,3
3,20230616_RCC_6731_pfa_OD_Hoechst-tub-nhs_series03,rcc6731_pfa_dna-tub-nhs_20230616_od_s03,NaN,3
4,20230616_RCC_6731_pfa_OD_Hoechst-tub-nhs_series04,rcc6731_pfa_dna-tub-nhs_20230616_od_s04,NaN,3
...,...,...,...,...
1762,vf21108_pfa_dna-tub-cetn_20251803_cl_s04,vf21108_pfa_dna-tub-cetn_20251803_cl_04,NaN,3
1763,vf21108_pfa_tub-cetn-dna_20251803_cl_s01,vf21108_pfa_tub-cetn-dna_20251803_cl_01,NaN,3
1764,vf21108_pfa_tub-cetn-dna_20251803_cl_s02,vf21108_pfa_tub-cetn-dna_20251803_cl_02,NaN,3
1765,vf21108_pfa_tub-cetn_20251803_cl_s03,vf21108_pfa_tub-cetn_20251803_cl_03,NaN,2


In [14]:
for view_name in table["new_filename_iteration4_manualRename"].to_list():
    print(view_name)

rcc6731_pfa_dna-tub-nhs_20230616_od_s00
rcc6731_pfa_dna-tub-nhs_20230616_od_s01
rcc6731_pfa_dna-tub-nhs_20230616_od_s02
rcc6731_pfa_dna-tub-nhs_20230616_od_s03
rcc6731_pfa_dna-tub-nhs_20230616_od_s04
rcc10669_pfa_cetn-tub-dna_20230803_cl_s00
rcc10669_pfa_cetn-tub-dna_20230803_cl_s01
rcc10669_pfa_cetn-tub-dna_20230803_cl_s02
rcc10669_pfa_cetn-tub-dna_20230803_cl_s03
rcc10669_pfa_cetn-tub-dna_20230803_cl_s04
rcc10669_pfa_cetn-tub-dna_20230803_cl_s05
rcc2962_pfa_dna-nhs-tub_20230516_cl_s00
rcc2962_pfa_dna-nhs-tub_20230516_cl_s01
rcc2962_pfa_dna-nhs-tub_20230516_cl_s02
rcc2962_pfa_dna-nhs-tub_20230516_cl_s03
rcc2962_pfa_dna-nhs-tub_20230516_cl_s04
rcc2962_pfa_dna-nhs-tub_20230516_cl_s05
rcc2962_pfa_dna-nhs-tub_20230516_cl_s06
rcc2962_pfa_dna-nhs-tub_20230516_cl_s07
rcc2962_pfa_dna-nhs-tub_20230516_cl_s08
rcc2962_pfa_dna-nhs-tub_20230516_cl_s09
rcc4630_pfa_dna-tub-nhs_20230802_od_s00
rcc4630_pfa_dna-tub-nhs_20230802_od_s01
rcc2962_pfa_cetn-tub-dna_20240604_cl_s00
rcc2962_pfa_cetn-tub-dna_20

In [21]:
len(table["new_filename_iteration4_manualRename"].tolist())

1767

In [36]:
# iterate over views
file_names: list[str] = []
series: list[str] = []
for view_name in table["new_filename_iteration4_manualRename"].tolist():
    if view_name == "default":
        continue
    # add unused files to the remove list
    if not isinstance(view_name, str):
        assert np.isnan(view_name)  # type: ignore
        file_names.append("")
        series.append("")
        continue
    if view_name == "delete":
        file_names.append("")
        series.append("")
        continue
    if "DROPFILE" in view_name:
        file_names.append("")
        series.append("")
        continue
    if (
        table.loc[table["new_filename_iteration4_manualRename"] == view_name, "remove"].values[0] == "x"  # type: ignore
    ):
        file_names.append("")
        series.append("")
        continue
    assert isinstance(view_name, str), f"{view_name} is not a string"
    # iterate over all sourceDisplays
    file_names_per_source: list[str] = []
    series_per_source: list[str] = []
    for i, source_display in enumerate(dataset["views"][view_name]["sourceDisplays"]):
        in_if = False
        # get old file path
        source_name = source_display["imageDisplay"]["sources"][0]
        assert source_name in dataset["sources"], f"{source_name} not in dataset"
        source = dataset["sources"][source_name]
        rel_path = source["image"]["imageData"]["ome.zarr"]["relativePath"].split("/")
        if rel_path[2] not in file_names_per_source:
            file_names_per_source.append(rel_path[2])
        if rel_path[3] not in series_per_source:
            series_per_source.append(rel_path[3])
    file_names.append(";".join(file_names_per_source))
    assert len(series_per_source) == 1, "Series names are not the same for all sources"
    series.append(series_per_source[0])

In [37]:
len(file_names), len(series)

(1767, 1767)

In [38]:
for f in file_names:
    print(f)

20230616_RCC_6731_pfa_OD_Hoechst-tub-nhs.ome.zarr
20230616_RCC_6731_pfa_OD_Hoechst-tub-nhs.ome.zarr
20230616_RCC_6731_pfa_OD_Hoechst-tub-nhs.ome.zarr
20230616_RCC_6731_pfa_OD_Hoechst-tub-nhs.ome.zarr
20230616_RCC_6731_pfa_OD_Hoechst-tub-nhs.ome.zarr
20230803_rccp22-88_pfa_centr-tub-hoechst.ome.zarr
20230803_rccp22-88_pfa_centr-tub-hoechst.ome.zarr
20230803_rccp22-88_pfa_centr-tub-hoechst.ome.zarr
20230803_rccp22-88_pfa_centr-tub-hoechst.ome.zarr
20230803_rccp22-88_pfa_centr-tub-hoechst.ome.zarr
20230803_rccp22-88_pfa_centr-tub-hoechst.ome.zarr
20230516_rcc2962_pfa_hoechst-nhs-tub.ome.zarr
20230516_rcc2962_pfa_hoechst-nhs-tub.ome.zarr
20230516_rcc2962_pfa_hoechst-nhs-tub.ome.zarr
20230516_rcc2962_pfa_hoechst-nhs-tub.ome.zarr
20230516_rcc2962_pfa_hoechst-nhs-tub.ome.zarr
20230516_rcc2962_pfa_hoechst-nhs-tub.ome.zarr
20230516_rcc2962_pfa_hoechst-nhs-tub.ome.zarr
20230516_rcc2962_pfa_hoechst-nhs-tub.ome.zarr
20230516_rcc2962_pfa_hoechst-nhs-tub.ome.zarr
20230516_rcc2962_pfa_hoechst-nhs-tub

In [39]:
for s in series:
    print(s)

0
1
2
3
4
0
1
2
3
4
5
0
1
2
3
4
5
6
7
8
9
0
1
0
1
2
3
0
1
2
0
1
0
1
0
1
2
0
1

3
4
5

7

9
0
1
2

1
2

4
5
6
7
8

1

3

5

7
8
9

11

13
0
1
2
0
0
1
2
0
0
1
2
3
4

1

3
0
1


0
0
1
0
1
0
1
0
0
0
1

3
4
5
6
7
8
9
0
1
2
3
4
0
1
2
3
4
0
1
2
0
1
2
3
0
1
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
3
4
5





5






12
13
14
15
16
17
0
1
2
3
4
0
1
0
1
2
0
1
2
3
4
0
0
0
0

1

3
4
5
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
0
1

1

0
1
0
1
2
3
0
1
2
0
1
2
0
0
0
0
0
0
0
1
2
3
0
1
2
3
4
5
6
7
8

1

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
0
0
1
2
0
1
2
3
0
1
2
3
4
5
6
7
0
1

3
4
5

7

9

11
0
1
2
3
0
1
0
0
1
0
1
0
0
0
0
0
0
0
1
2
3

1

3

5

7

9
10
11
12
13
0
1
2
0
0
0
1
2
3
0
1
2
0
1
0
1
2
0
1
2
3
4
5
6
7
8
9
10
11
12
13
0
1
2
3
4
0
1
2
0
0
0
0
0
0
0

0
0
0
0
0
0
0
0
0
0


2

4

6
7
8
9
10
11
12
13
14
0
1
2
3
0
1
2
3
4
5
6
7
8
9

1

1
0
1
0
1
0
1
0
1
2
3
4
5
6
0
0
0
0
0
1
2
3
4
5
0
1

3
0
0
0
0
0
0
0
0
0
0
0
1
2
3
4
0
0
0
0
0
1
2
3
4
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
2
3
4
0
1
2